In [1]:
import sys
sys.path.append('..')

import json
import warnings
import polars as pl
from estimator import*
from QHFactor.fn import *

from features_selector import*
from secuntial import Sequential


from QHFactor.transformer import Load_X_y
from datetime import datetime,timedelta
from alphalens import tears,utils,performance
from QHData.data_load import DataLoader,CodesLoader

from sklearn.model_selection import train_test_split

warnings.filterwarnings('ignore')

No module named 'wtpy'
Error importing QUANTAXIS module: No module named 'QUANTAXIS'


In [2]:
# 1.Get dataset
seq = Sequential(name='STKIDX_LGBMREG_60M')

days = 200
start_date = (datetime.now() - timedelta(days=days)).strftime('%Y-%m-%d')
end_date = datetime.now().strftime('%Y-%m-%d')

dataset, returns = seq.data_loader(start_date, end_date, '60min')
# data_loader = DataLoader(start_date,end_date,'d')

# dataset,returns = data_loader.load_future()

# 重新构建标签
# 2.calculate factors
# X = dataset.to_pandas().set_index(['datetime','code']).sort_index()
# y = returns.to_pandas().set_index(['datetime','code']).sort_index()
# factors = seq.calculate_factors(X, window=60).fillna(0)

# 3.select features
# classifi_labels = pd.qcut(x
# lable['returns'], q=10, labels=list(range(0, 10))).astype(int)

# # rank lable
# rank_lables = lable.groupby(level=0).rank('dense',ascending=False).astype(int)

# X, y = Load_X_y().transform(factors,lable.iloc[:,0])
# X_train, X_test, y_train, y_test = train_test_split(factors, y.iloc[:,0], test_size=0.1, shuffle=False)
# 
# features = seq.select_features(LGBMFeatureSelector(model_type='reg', n=20), X_train, y_train)
# features = seq.select_features(RFEFeaturesSelector(model_type='reg',n=20), X_train, y_train)

[2025-03-12 11:12:10 - INFO] Model name: STKIDX_LGBMREG_60M
[2025-03-12 11:12:10 - INFO] Training started at 2025-03-12 11:12:10
[2025-03-12 11:12:10 - INFO] # 1. Get dataset.
[2025-03-12 11:12:10 - INFO] Data loading...
[2025-03-12 11:16:31 - INFO] Codes: ['RBL8', 'SAL8', 'ML8', 'FGL8', 'AGL8', 'RML8', 'VL8', 'PL8', 'TAL8', 'MAL8']
[2025-03-12 11:16:31 - INFO] Dataset from 2024-08-24 to 2025-03-12
[2025-03-12 11:16:31 - INFO] Frequency: 60min
[2025-03-12 11:16:31 - INFO] Dataset shape: (7972, 6)
[2025-03-12 11:16:31 - INFO] Data loading completed


In [10]:
# dataset.loc[(slice(None),'RBL8'),]
dataset.index.levels[1]

Index(['AGL8', 'FGL8', 'MAL8', 'ML8', 'PL8', 'RBL8', 'RML8', 'SAL8', 'TAL8',
       'VL8'],
      dtype='object', name='code')

In [10]:
df = DataLoader(start_date=start_date,end_date=end_date,freq='60min').load_future(codes=['RBL8','AGL8'])

In [ ]:
# 4.Train model
model = seq.train_model(LGBMRollingRegressor,features, y_train)

In [ ]:
# 6.track model
config_data = {
    'codes': features.index.levels[1].unique().tolist(),
    'features': features.columns.tolist()
}

local_uri = 'http://192.168.215.3:5115'
remote_uri = 'http://192.168.31.220:5115'

seq.track_model(
    track_ui=local_uri,
    exp_name=seq.name,
    model=model,
    params=model.get_params(),
    # metrics=perf_all,
    config=config_data,
    tags={'author':'chiang','version':'1.0'},
    # image=f'{seq.name}_performence.png'
)


In [ ]:
import numpy as np
from QHMlflow.mlflowtracker import load_model_and_config

# 假设您已经有了以下数据
# 1. 预测结果
# model,config = load_model_and_config(remote_tracking_uri='',run_id='')

predictions = pd.Series(index=X.index, data=model.predict(X.loc[:, features.columns]))

# 2. 价格数据
# 假设 dataset 是一个包含价格数据的 DataFrame，包含 'close' 列
prices = dataset.loc[X.index,'close'].unstack()  # 将价格数据转换为适合 AlphaLens 的格式

# 3. 计算因子和前向收益
factor_data = utils.get_clean_factor_and_forward_returns(
    quantiles=10,
    factor=predictions,
    prices=dataset.loc[predictions.index,'close'].unstack(),
    periods=(1, 5,10)  # 1天、5天和10天的前向收益
)

# 4. 分析因子表现
# 计算因子表现
# tears = tears.create_full_tear_sheet(factor_data)
factor_data.loc[factor_data.factor_quantile==10,'1D'].groupby(level=0).mean().cumsum().plot()

In [ ]:
# 7、刪除試驗環境
import mlflow
# mlflow.delete_experiment(experiment_id='377574715235855307')
mlflow.delete_run(run_id='')



In [ ]:
import sys
sys.path.append('..')
from QHFactor.fn import *
from QHData.data_load import DataLoader

data_loader = DataLoader(start_date='2024-01-01',end_date='2025-01-02',freq='weekly')
data_loader.load_future(codes=['IFL8','IHL8','ICL8','IML8'])